In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns


from matplotlib import pyplot as plt

In [ ]:
# program = "graph_coloring"
# program = "maximal_matching"
program = "dijkstra_token_ring"
# graph_name = "graph_7"
graph_name = "implicit_graph_n10"

no_simulations = 100000
fault_interval = 3
sched = 0

selected_nodes = [0, 1, 9]

simulation_type = "controlled_at_node_amit_v2"
limits = 100

merge_files = [
    f"w_exp_{program}__{graph_name}__{sched}__{no_simulations}__{fault_interval}__{n}__{simulation_type}__{limits}"
    for n in selected_nodes
]

In [3]:
dfs = [pd.read_csv(f"{os.path.join("results", program, f)}.csv").drop('Count', axis=1) for f in merge_files]
for df in dfs:
    df['Steps'] = np.ceil(df['Steps']).astype(int)

In [4]:
dfs[2]

,Steps,exp_fit
0,0,2764.373985
1,1,2722.668810
2,2,2681.172240
3,3,2639.883233
4,4,2598.800749
...,...,...
95,95,-395.411387
96,96,-421.311623
97,97,-447.082309
98,98,-472.724093


In [5]:
dfs[0]

,Steps,exp_fit
0,0,2485.896092
1,1,2451.636724
2,2,2417.520629
3,3,2383.547209
4,4,2349.715866
...,...,...
95,95,-204.555696
96,96,-227.563521
97,97,-250.475127
98,98,-273.290915


In [6]:
dfs[1]

,Steps,exp_fit
0,0,2064.435746
1,1,2044.528990
2,2,2024.576066
3,3,2004.576869
4,4,1984.531290
...,...,...
95,95,-48.486052
96,96,-73.293002
97,97,-98.157483
98,98,-123.079629


In [7]:
df_merged = dfs[0].copy()
for i in range(1, len(dfs)):
    df_merged = pd.merge(
        df_merged,
        dfs[i],
        on=["Steps"],
        suffixes=(f"__{selected_nodes[i - 1]}", ""),
    ).fillna(0)

df_merged.rename(columns={df_merged.columns[-1]: f'exp_fit__{selected_nodes[-1]}'}, inplace=True)
df_merged = df_merged.set_index('Steps')
df_merged

,exp_fit__0,exp_fit__1,exp_fit__9
Steps,,,
0,2485.896092,2064.435746,2764.373985
1,2451.636724,2044.528990,2722.668810
2,2417.520629,2024.576066,2681.172240
3,2383.547209,2004.576869,2639.883233
4,2349.715866,1984.531290,2598.800749
...,...,...,...
95,-204.555696,-48.486052,-395.411387
96,-227.563521,-73.293002,-421.311623
97,-250.475127,-98.157483,-447.082309


In [8]:
for col in df_merged.columns:
    df_merged.rename(columns={col: f"Exponential fit of node {col.split('__')[1]}"}, inplace=True)

df_merged

,Exponential fit of node 0,Exponential fit of node 1,Exponential fit of node 9
Steps,,,
0,2485.896092,2064.435746,2764.373985
1,2451.636724,2044.528990,2722.668810
2,2417.520629,2024.576066,2681.172240
3,2383.547209,2004.576869,2639.883233
4,2349.715866,1984.531290,2598.800749
...,...,...,...
95,-204.555696,-48.486052,-395.411387
96,-227.563521,-73.293002,-421.311623
97,-250.475127,-98.157483,-447.082309


In [9]:
fontsize = 20

def get_title():
    return f"Simulation curve fit - {program} | {graph_name} | Sched: {sched} | N: {no_simulations:,} | FI: {fault_interval}"


In [10]:
# sns.lineplot(df_merged, dashes={f"Exponential fit of node {i}":"" for i in selected_nodes}, linewidth=1)

In [11]:
plt.figure(figsize=(16, 8))
ax = sns.lineplot(data=df_merged, dashes={f"Exponential fit of node {i}":"" for i in selected_nodes}, linewidth=1)
ax.set_title(get_title(), fontdict={"fontsize": fontsize})

ax.tick_params(axis="x", labelsize=fontsize)
ax.tick_params(axis="y", labelsize=fontsize)
ax.xaxis.label.set_size(fontsize)
ax.yaxis.label.set_size(fontsize)


ax.set_xlabel("Steps")
ax.set_ylabel("Count")

plt.rc("font", size=fontsize)
plt.legend(fontsize=fontsize * 0.9)

plt.savefig(
    f"exp_curve_fit_{program}__{graph_name}__{''.join([str(i) for i in selected_nodes])}__{fault_interval}__{limits}.png",
    bbox_inches="tight",
)
plt.close()